In [ ]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='en-US')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u'Transcript: {}'.format(result.alternatives[0].transcript))

In [11]:
from __future__ import division

import re
import sys

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pyaudio
from six.moves import queue

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)





    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    


if __name__ == '__main__':
    language_code = 'en-US'  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code)
    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (types.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_loop(responses)

num_chars_printed = 0
for response in responses:
    if not response.results:
        continue

    # The `results` list is consecutive. For streaming, we only care about
    # the first result being considered, since once it's `is_final`, it
    # moves on to considering the next utterance.
    result = response.results[0]
    if not result.alternatives:
        continue

    # Display the transcription of the top alternative.
    transcript = result.alternatives[0].transcript

    # Display interim results, but with a carriage return at the end of the
    # line, so subsequent lines will overwrite them.
    #
    # If the previous result was longer than this one, we need to print
    # some extra spaces to overwrite the previous result
    overwrite_chars = ' ' * (num_chars_printed - len(transcript))

    if not result.is_final:
        sys.stdout.write(transcript + overwrite_chars + '\r')
        sys.stdout.flush()

        num_chars_printed = len(transcript)

    else:
        print(transcript + overwrite_chars)

        # Exit recognition if any of the transcribed phrases could be
        # one of our keywords.
        if re.search(r'\b(exit|quit)\b', transcript, re.I):
            print('Exiting..')
            break

        num_chars_printed = 0




hello
 this is so exciting
 oh my God quit
Exiting..


In [16]:
print(u'Transcript: {}'.format(result.alternatives[0].transcript))

NameError: name 'result' is not defined

In [5]:
dir(responses)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_wrapped',
 'add_callback',
 'cancel',
 'code',
 'details',
 'initial_metadata',
 'is_active',
 'next',
 'time_remaining',
 'trailing_metadata']

In [2]:
#google translation api
from google.cloud import translate

translate_client = translate.Client()

text = responses
target = 'fr'
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))


TypeError: Object of type '_StreamingResponseIterator' is not JSON serializable

In [8]:
#text to speech api
new_text=translation['translatedText']
import pyttsx3
engine = pyttsx3.init()
engine.say(new_text)
engine.runAndWait()